In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset, random_split
import os
import zipfile
import shutil
import json
import pandas as pd
import time
from tqdm import tqdm

OUTPUT_DIR = "/kaggle/working/experiment_results"
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "weights")
LOGS_DIR = os.path.join(OUTPUT_DIR, "logs")

os.makedirs(WEIGHTS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

EPOCHS = 40
BATCH_SIZE = 256
LEARNING_RATE = 0.01
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f" Device: {DEVICE}")
print(f" Output Dir: {OUTPUT_DIR}")

 Device: cuda
 Output Dir: /kaggle/working/experiment_results


In [2]:
def prepare_cats_dogs(input_zip_path, extract_to):

    if os.path.exists(extract_to):
        print("Dataset already extracted.")
        return

    print("Extracting Cats vs Dogs dataset")
    with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(extract_to))
    
    source_dir = os.path.join(os.path.dirname(extract_to), 'train')
    target_dir = extract_to
    
    os.makedirs(os.path.join(target_dir, 'cat'), exist_ok=True)
    os.makedirs(os.path.join(target_dir, 'dog'), exist_ok=True)
    
    files = os.listdir(source_dir)
    for file in tqdm(files, desc="Organizing Images"):
        if file.startswith('cat'):
            shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, 'cat', file))
        elif file.startswith('dog'):
            shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, 'dog', file))

    os.rmdir(source_dir)

KAGGLE_INPUT_ZIP = "/kaggle/input/dogs-vs-cats/train.zip"
PROCESSED_DATA_DIR = "/kaggle/working/data/cats_dogs_sorted"

has_cats_dogs = False
if os.path.exists(KAGGLE_INPUT_ZIP):
    prepare_cats_dogs(KAGGLE_INPUT_ZIP, PROCESSED_DATA_DIR)
    has_cats_dogs = True
else:
    print("'dogs-vs-cats' dataset not found in input. Skipping.")

transform_cifar = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_cd = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Loaders
print("\nLoading CIFAR-10")
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
loader_cifar_train = DataLoader(cifar_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=2)
loader_cifar_test = DataLoader(cifar_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=2)

loader_cd_train = None
loader_cd_test = None

if has_cats_dogs:
    print("Loading Cats vs Dogs")
    full_cd = torchvision.datasets.ImageFolder(root=PROCESSED_DATA_DIR, transform=transform_cd)
    train_size = int(0.8 * len(full_cd))
    test_size = len(full_cd) - train_size
    cd_train, cd_test = random_split(full_cd, [train_size, test_size])
    
    loader_cd_train = DataLoader(cd_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=2)
    loader_cd_test = DataLoader(cd_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=2)

Extracting Cats vs Dogs dataset


Organizing Images: 100%|██████████| 25000/25000 [00:00<00:00, 40483.40it/s]



Loading CIFAR-10


100%|██████████| 170M/170M [00:04<00:00, 34.4MB/s]


Loading Cats vs Dogs


In [3]:
class FlexibleCNN(nn.Module):
    def __init__(self, activation='relu', init_method='xavier', num_classes=10):
        super(FlexibleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.global_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)
        
        if activation == 'tanh':
            self.act_fn = nn.Tanh()
        elif activation == 'leaky_relu':
            self.act_fn = nn.LeakyReLU(0.1)
        else:
            self.act_fn = nn.ReLU()
            
        self._init_weights(init_method)

    def _init_weights(self, method):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                if method == 'xavier':
                    nn.init.xavier_uniform_(m.weight)
                elif method == 'kaiming':
                    nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif method == 'random':
                    nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.pool(self.act_fn(self.bn1(self.conv1(x))))
        x = self.pool(self.act_fn(self.bn2(self.conv2(x))))
        x = self.pool(self.act_fn(self.bn3(self.conv3(x))))
        
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.act_fn(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [4]:
def run_experiment(dataset_name, train_loader, test_loader, num_classes):
    configs = []
    
    activations = ['relu', 'tanh', 'leaky_relu']
    inits = ['xavier', 'kaiming', 'random']
    optimizers = ['sgd', 'adam', 'rmsprop']
    
    total_configs = len(activations) * len(inits) * len(optimizers)
    current_config = 0

    print(f"Starting Experiment on {dataset_name} ({total_configs} Configurations)")

    for act in activations:
        for init in inits:
            for opt_name in optimizers:
                current_config += 1
                config_id = f"{dataset_name}_{act}_{init}_{opt_name}"
                print(f"\n[{current_config}/{total_configs}] Running: {config_id}")
                
                model = FlexibleCNN(act, init, num_classes).to(DEVICE)
                
                if opt_name == 'sgd':
                    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
                elif opt_name == 'adam':
                    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
                elif opt_name == 'rmsprop':
                    optimizer = optim.RMSprop(model.parameters(), lr=LEARNING_RATE)
                
                criterion = nn.CrossEntropyLoss()
                
                start_time = time.time()
                best_acc = 0.0
                
                for epoch in range(EPOCHS):
                    model.train()
                    for imgs, labels in train_loader:
                        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                        optimizer.zero_grad()
                        loss = criterion(model(imgs), labels)
                        loss.backward()
                        optimizer.step()
                
                duration = time.time() - start_time
                
                model.eval()
                correct, total = 0, 0
                with torch.no_grad():
                    for imgs, labels in test_loader:
                        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(imgs)
                        _, predicted = torch.max(outputs, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                
                final_acc = 100 * correct / total
                print(f"   -> Finished. Accuracy: {final_acc:.2f}% | Time: {duration:.1f}s")
        
                weights_path = os.path.join(WEIGHTS_DIR, f"{config_id}.pth")
                torch.save(model.state_dict(), weights_path)
                
                config_data = {
                    "dataset": dataset_name,
                    "activation": act,
                    "initialization": init,
                    "optimizer": opt_name,
                    "epochs": EPOCHS,
                    "final_accuracy": final_acc,
                    "training_time_sec": duration,
                    "weights_path": weights_path
                }
                configs.append(config_data)
         
                pd.DataFrame(configs).to_csv(os.path.join(LOGS_DIR, f"results_{dataset_name}.csv"), index=False)

    return pd.DataFrame(configs)

In [5]:
cifar_results = run_experiment("CIFAR10", loader_cifar_train, loader_cifar_test, num_classes=10)

if loader_cd_train:
    cd_results = run_experiment("CatsDogs", loader_cd_train, loader_cd_test, num_classes=2)

print(f"Results saved to: {LOGS_DIR}")

Starting Experiment on CIFAR10 (27 Configurations)

[1/27] Running: CIFAR10_relu_xavier_sgd
   -> Finished. Accuracy: 79.10% | Time: 300.4s

[2/27] Running: CIFAR10_relu_xavier_adam
   -> Finished. Accuracy: 78.92% | Time: 298.6s

[3/27] Running: CIFAR10_relu_xavier_rmsprop
   -> Finished. Accuracy: 73.31% | Time: 298.5s

[4/27] Running: CIFAR10_relu_kaiming_sgd
   -> Finished. Accuracy: 77.10% | Time: 302.6s

[5/27] Running: CIFAR10_relu_kaiming_adam
   -> Finished. Accuracy: 76.92% | Time: 302.1s

[6/27] Running: CIFAR10_relu_kaiming_rmsprop
   -> Finished. Accuracy: 71.45% | Time: 299.5s

[7/27] Running: CIFAR10_relu_random_sgd
   -> Finished. Accuracy: 82.43% | Time: 296.0s

[8/27] Running: CIFAR10_relu_random_adam
   -> Finished. Accuracy: 75.20% | Time: 299.2s

[9/27] Running: CIFAR10_relu_random_rmsprop
   -> Finished. Accuracy: 72.69% | Time: 299.1s

[10/27] Running: CIFAR10_tanh_xavier_sgd
   -> Finished. Accuracy: 75.34% | Time: 298.5s

[11/27] Running: CIFAR10_tanh_xavier_ad

In [6]:
def train_resnet(dataset_name, train_loader, test_loader, num_classes):
    print(f"\nTraining ResNet-18 on {dataset_name}...")
    
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model = model.to(DEVICE)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-4) # Lower LR for fine-tuning
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(EPOCHS):
        model.train()
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{EPOCHS} completed.")

   
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    acc = 100 * correct / total
    print(f"ResNet-18 {dataset_name} Accuracy: {acc:.2f}%")

    save_path = os.path.join(WEIGHTS_DIR, f"resnet18_{dataset_name}.pth")
    torch.save(model.state_dict(), save_path)
    return acc

resnet_cifar_acc = train_resnet("CIFAR10", loader_cifar_train, loader_cifar_test, 10)

if loader_cd_train:
    resnet_cd_acc = train_resnet("CatsDogs", loader_cd_train, loader_cd_test, 2)


Training ResNet-18 on CIFAR10...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 208MB/s]


Epoch 10/40 completed.
Epoch 20/40 completed.
Epoch 30/40 completed.
Epoch 40/40 completed.
ResNet-18 CIFAR10 Accuracy: 83.03%

Training ResNet-18 on CatsDogs...
Epoch 10/40 completed.
Epoch 20/40 completed.
Epoch 30/40 completed.
Epoch 40/40 completed.
ResNet-18 CatsDogs Accuracy: 97.54%


In [7]:
df = pd.read_csv(os.path.join(LOGS_DIR, "results_CIFAR10.csv"))
best_row = df.loc[df['final_accuracy'].idxmax()]

print(f"Best Custom CNN Configuration: {best_row['activation']} + {best_row['initialization']} + {best_row['optimizer']}")
print(f"Best Custom CNN Accuracy: {best_row['final_accuracy']:.2f}%")
print(f"ResNet-18 Accuracy: {resnet_cifar_acc:.2f}%")
print(f"\nAll weights saved in: {WEIGHTS_DIR}")
print(f"All logs saved in: {LOGS_DIR}")

Best Custom CNN Configuration: relu + random + sgd
Best Custom CNN Accuracy: 82.43%
ResNet-18 Accuracy: 83.03%

All weights saved in: /kaggle/working/experiment_results/weights
All logs saved in: /kaggle/working/experiment_results/logs
